In [2]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from json import load
import os
import tgt

# To Do
- include bb and rexp
- display buttons in visualization
- create multicolor lines (https://matplotlib.org/examples/pylab_examples/multicolored_line.html)
    - Zeit durch Alpha-Wert codieren
- decrease image saturation to improve line visibility
- verweildauer (zeit bis zum nächsten Datenpunkt) codieren - Größe des Punktes?

In [100]:
def find_file(suffix, path):
    """
    return the filename of the log file
    """
    result_files = []
    for root, dirs, files in os.walk(path):
        for file in [f for f in files if f.endswith(suffix)]:
            result_files.append(file)
    return result_files
        
def read_file(path):
    """
    return content of log file
    """
    with open(path) as raw_jfile:
        content = load(raw_jfile)
        return content
    
def crop_list(lst):
    """
    remove entries from log file before the game starts and after the game is finished
    return cropped file as list
    """
    sub_lst = []
    for entry in lst:
        try:
            if entry["msg"] == "Game started!" and entry["user"]["name"]=="Image_Click_Bot":
                sub_lst = lst[lst.index(entry):]
            if "no images left" in entry["msg"].lower() and entry["user"]["name"]=="Image_Click_Bot":
                sub_lst = lst[:lst.index(entry)+1]
        except:
            pass
    return sub_lst

def json_filename(file):
    for entry in file: 
        try:
            if "json file:" in entry["msg"]:
                return (entry["msg"].split()[-1])
        except:
            pass

def images_in_log(file):
    """
    split log file on new_image commands
    return list containing one section for each image in log file, 
    each of them starting with a new_image command
    """
    file = crop_list(file)
    images = []
    i = False
    i_prev = False
    for entry in [e for e in file if e["type"] == "new_image"]:
        if i:
            i_prev = i
            i = file.index(entry)
            images.append(file[i_prev:i])
        else:
            i = file.index(entry)
    images.append(file[i:]) # slice from last new_image to end of file
    return images

def image_data(img_slice):
    """
    extract image filename and tracking coordinates from img section in log file
    """
    buttons = ["#reportButton", "#replayButton"]
    coordinates = []
    clicks=[]
    button_clicks = []
    filename = ""
    for entry in img_slice: 
        try: 
            if entry["type"]=="new_image":
                filename = entry["url"].split("/")[-1]
            if entry["type"]=="mouse_positions":
                coordinates = entry["data"]
            if entry["type"]=="mouse_click" and entry["element"]=="#current-image":
                clicks.append(entry["coordinates"])
            if entry["type"]=="mouse_click" and entry["element"] in buttons:
                button_clicks.append(entry["coordinates"])
        except: 
            pass
    return filename,coordinates,clicks,button_clicks

In [4]:
cwd = os.getcwd()
cwd = "/home/simeon/Dokumente/mouse-tracking-experiment/evaluation/"
os.chdir(cwd+"/data/")
files = find_file(".log", os.getcwd())
print (files)
log_file = read_file(files[0])
json = json_filename(log_file)
os.chdir(cwd)

['2018-12-17 16-23-26-meetup 1.log']


In [156]:
os.chdir(cwd+"/data/")

json_content = read_file("json/"+json)
paths = []
for entry in json_content:
    try:
        audio_wav = json_content[entry]["audio_filename"]
        audio_json = os.path.splitext(audio_wav)[0]+".json"
        img_path = json_content[entry]["image_filename"]
        paths.append({'audio_wav':audio_wav, 'audio_json': audio_json, 'img_path': img_path})
    except:
        pass

imgs = images_in_log(log_file)
imgs_with_paths = list(zip(imgs, paths))[:1] # only 1st element in list
for entry in imgs_with_paths:
    log_data, files = entry
    for e in log_data:
        if e["type"]=="mouse_click" and e["data"]["element"] == "#overlayButton":
            time_base = e["data"]["timestamp"]
            print ("time base:",time_base)
    
        if e["type"]=="mouse_positions": 
            mouse_positions = e["data"]
            
for entry in log_data[::-1]:
    if entry["type"] == "mouse_click":
        timestamp_end = entry["data"]["timestamp"]
        break

img_duration = (timestamp_end - time_base)+100

mouseclicks = []

for entry in log_data:
    if entry["type"] == "mouse_click":
        mouseclicks.append(((entry["data"]["timestamp"]-time_base), entry["data"]["element"]))

#for entry in mouse_positions: 
    #print (entry["timestamp"])
    #print (entry["timestamp"]-time_base)
#    pass
    
os.chdir(cwd)

time base: 1545060211877


In [157]:
# https://textgridtools.readthedocs.io/en/stable/api.html

with open("test.textgrid", "w") as tg:
    tg.write('File type = "ooTextFile"\nObject class = "TextGrid"\n')
    
textgrid = tgt.core.TextGrid(filename='test.textgrid')

clicks =  tgt.core.PointTier(start_time=0, end_time=img_duration/1000, name='clicks', objects=None)
movement = tgt.core.PointTier(start_time=0, end_time=img_duration/1000, name='movement', objects=None)
utterances = tgt.core.IntervalTier(start_time=0, end_time=img_duration/1000, name='utterances', objects=None)

textgrid.add_tiers((clicks, movement, utterances))

tgt.write_to_file(textgrid, "test.textgrid", format='short')

In [158]:
for entry in mouseclicks:
    clicks.add_point(tgt.core.Point(entry[0]/1000, text=entry[1]))

In [159]:
tgt.write_to_file(textgrid, "test.textgrid", format='short')

In [7]:
cwd = "/home/simeon/Dokumente/mouse-tracking-experiment/evaluation/"
os.chdir(cwd)